## Day 19: Beacon Scanner

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/mazharenko/AoC-2021/tree/HEAD/notebooks/day19/puzzle.ipynb)

In [ ]:

#load "../common.fsx"
#load "./rotation.fsx"
#r "nuget: MathNet.Numerics.FSharp, 5.0.0-alpha03"
open MathNet.Numerics.LinearAlgebra
open System.Collections.Generic


type Point = int*int*int
type BeaconAlts = { Beacon: Point; AllRelative: HashSet<Point>; Shift: Point }
type BaseScannerAltData = { BeaconAlts: BeaconAlts[] }
type SecondaryScannerAltData = { AxisAlts: (Rotation*BeaconAlts[])[] }
type UnsolvedScannerData = { OriginalBeacons: HashSet<Point>; AltData: SecondaryScannerAltData }
type SolvedScannerData = { Position: Point; Beacons: HashSet<Point>; AltData : BaseScannerAltData}


let rotate (rotation: Rotation) ((x,y,z): Point) : Point =
    let rotated = (vector [float x; float y; float z]) * rotation 
    (int rotated.[0], int rotated.[1], int rotated.[2])
    

Installed Packages MathNet.Numerics.FSharp, 5.0.0-alpha03

In [ ]:
let (.-) (x1, y1, z1) (x2, y2, z2) =
    (x1-x2, y1-y2, z1-z2)
    
let (.+) (x1, y1, z1) (x2, y2, z2) =
    (x1+x2, y1+y2, z1+z2)

In [ ]:
let parsePoints (input: string) = 
    input.Split("\r\n\r\n", StringSplitOptions.RemoveEmptyEntries)
    |> Array.map (fun block ->
                    let points = 
                        block.Split("\r\n", StringSplitOptions.RemoveEmptyEntries) |> Array.skip 1
                        |> Array.map (fun line ->
                            let [x;y;z] = line.Split(",") |> Array.toList |> List.map int
                            Point(x,y,z)
                        ) |> HashSet
                    points
                )

let baseAlts point points =
    let shift = (0,0,0) .- point
    let pointsRebased = points |> Seq.map (fun p -> p .+ shift)
    {Beacon = point; AllRelative = pointsRebased |> HashSet; Shift = shift}
    
let secondaryAlts points =
    let alts = 
        rotations90
        |> Array.map (
            fun rotation ->
                let rotatedPoints = points |> Array.map (rotate rotation)
                rotation,
                rotatedPoints |> Array.map (fun point -> baseAlts point rotatedPoints)
            )
    { AxisAlts = alts }

let prepareScanners (inputPoints : HashSet<Point>[]) = 
    
    let solvedScanner =
        let solvedPoints = inputPoints.[0]
        let baseAlts = solvedPoints |> Seq.map (fun point -> baseAlts point solvedPoints) |> Array.ofSeq
        {Position = 0,0,0; Beacons = solvedPoints; AltData = {BeaconAlts = baseAlts}} 
        
    let unsolvedScanners =
        inputPoints
        |> Array.skip 1
        |> Array.map (fun points ->
            let secAlts = secondaryAlts (points |> Seq.toArray)
            {OriginalBeacons = points; AltData = secAlts}
        )
        |> List.ofArray

    solvedScanner, unsolvedScanners

In [ ]:
#!value --name sampleRaw --from-file ./sample

In [ ]:
#!value --name inputRaw --from-file ./data

In [ ]:
#!share inputRaw --from value
#!share sampleRaw --from value
let sampleScanners = sampleRaw |> parsePoints |> prepareScanners
let actualScanners = inputRaw |> parsePoints |> prepareScanners

In [ ]:

let intersection set1 (set2 : HashSet<_>) =
    seq {
        for el1 in set1 do
            if (set2.Contains el1) then
                yield el1
    } |> Seq.length

let doesMatch (basePoints: BaseScannerAltData) (secondaryPoints: SecondaryScannerAltData) =
    let allBaseAlts = basePoints.BeaconAlts |> Seq.map (fun alts -> alts.Shift, alts.AllRelative)
    let allSecondaryAlts =
        secondaryPoints.AxisAlts
        |> Seq.collect (fun (rotation, alts) -> Array.map (fun a -> rotation, a.Shift, a.AllRelative) alts)
    let matches =
        (allBaseAlts, allSecondaryAlts)
        ||> Seq.allPairs
        |> Seq.map (fun ((baseShift, baseSet), (secondaryRotation, secondaryShift, secondarySet)) -> baseShift, secondaryShift, secondaryRotation,intersection baseSet secondarySet)
        |> Seq.filter(fun (_, _, _, intersection) -> intersection >= 12) 
        |> Seq.toArray
    
    if (Array.isEmpty matches)
    then None
    else 
        matches
        |> Array.map (fun (baseShift, secondaryShift, rotation, _) -> baseShift .- secondaryShift, rotation)
        |> Array.distinct
        |> Array.exactlyOne
        |> Some

In [ ]:
let rec solve solvedScanner unsolvedScanners = 
    solve1 [solvedScanner] unsolvedScanners
and private solve1 (solvedScanners : SolvedScannerData list) (unsolvedScanners : UnsolvedScannerData list) =
    match unsolvedScanners with
    | [] -> solvedScanners
    | unsolvedScanner::restUnsolved ->
        let matching = 
            solvedScanners
            |> Seq.choose (fun solved -> doesMatch solved.AltData unsolvedScanner.AltData)
            |> Seq.tryHead
        match matching with
        | None -> solve1 solvedScanners (restUnsolved @ [unsolvedScanner])
        | Some (shift, rotation) ->
            let newSolvedPoints =
                unsolvedScanner.OriginalBeacons
                |> Seq.map (rotate (rotation))
                |> Seq.map (fun point -> point .- shift)
                |> HashSet
                
            let newSolvedList =
                {
                    Position = (0,0,0) .- shift
                    Beacons = newSolvedPoints
                    AltData = {BeaconAlts = newSolvedPoints |> Seq.map (fun point -> baseAlts point newSolvedPoints) |> Array.ofSeq}
                }::solvedScanners
            solve1 newSolvedList restUnsolved

In [ ]:
let sampleSolved = sampleScanners ||> solve
let actualSolved = actualScanners ||> solve

In [ ]:
sampleSolved |> Seq.collect (fun solved -> solved.Beacons) |> Seq.distinct |> Seq.length |> display
actualSolved |> Seq.collect (fun solved -> solved.Beacons) |> Seq.distinct |> Seq.length |> display

79

362

In [ ]:

let manhattan (x1, y1, z1) (x2, y2, z2) =
    abs (x1 - x2) + abs (y1 - y2) + abs (z1 - z2)

In [ ]:
(sampleSolved, sampleSolved)
||> Seq.allPairs
|> Seq.map (fun (scanner1,scanner2) -> manhattan scanner1.Position scanner2.Position)
|> Seq.max 
|> display

(actualSolved, actualSolved)
||> Seq.allPairs
|> Seq.map (fun (scanner1,scanner2) -> manhattan scanner1.Position scanner2.Position)
|> Seq.max 
|> display

3621

12204